## Introduction to DataBase System

데이터베이스(Database)는 데이터를 체계적으로 저장하고 관리하는 시스템이다. 데이터를 구조화된 형태로 저장하여 효율적인 검색, 수정, 삭제가 가능하도록 돕는다. 데이터베이스는 크게 엑셀처럼 행과 열로 관리되는 관계형 데이터베이스(RDBMS)와 키와 밸류 형태로 관리되는 NoSQL 데이터베이스로 이뤄진다. 

그중 관계형 데이터베이스는 데이터를 테이블 형태로 저장하며, 각 테이블은 행(Row)과 열(Column)로 구성된다. 데이터베이스는 대량의 데이터를 안정적으로 관리하고, 여러 사용자와 애플리케이션이 동시에 데이터를 활용할 수 있게 한다. 관계형 데이터베이스는 특히 금융 데이터를 다루는 데 있어서 알맞은 형태로, 시계열 및 고객 데이터와 거래 기록들을 다루는 데 있어서 알맞게 사용가능하다. 최근에는 High Frequency 데이터 처리 및 market depth, order book 데이터 관리의 중요성이 대두되면서, NoSQL형 데이터도 증가하는 추세에 있다.

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

TICKERS = ['^SPX','^IXIC','^RUT','^DJI','^XAX']
START_DATE = '1990-01-01'

### 1. Structured Query Language

`SQL(Structured Query Language)`은 관계형 데이터베이스를 관리하기 위한 표준 언어이다. 1970년대 IBM에서 SEQUEL이라는 이름으로 처음 개발되었으며, 이후 ANSI와 ISO에 의해 표준화되었다. SQL은 관계형 데이터베이스에서 데이터를 표 형식으로 다루며, 데이터 간의 관계를 정의하고 관리하는 데 적합하다. 

SQL은 데이터 정의, 조작, 제어라는 세 가지 주요 기능으로 나뉘는데, **데이터 정의** 는 테이블과 같은 데이터베이스 구조를 생성, 수정, 삭제하는 작업으로, 주로 데이터베이스 설계 초기 단계에서 사용된다. **데이터 조작** 은 삽입, 조회, 수정, 삭제와 같은 데이터를 직접 다루는 작업으로, 데이터베이스의 일상적인 사용에서 가장 많이 쓰인다. **데이터 제어** 는 사용자 권한을 관리하고 데이터베이스의 보안을 설정하는 데 초점이 맞춰져 있다. 이러한 기능은 데이터베이스의 설계부터 운영까지 모든 단계에서 필수적이다. 예를 들어, 새로운 테이블을 생성하고 데이터를 삽입한 뒤, 특정 사용자에게만 접근 권한을 부여하는 방식으로 활용될 수 있다. SQL의 각 기능은 효율적인 데이터 관리를 가능하게 하며, 데이터 무결성과 보안을 보장한다.

SQL은 데이터를 정의하는 DDL(Data Definition Language), 데이터를 조작하는 DML(Data Manipulation Language), 데이터를 제어하는 DCL(Data Control Language)로 구성되어 있다. 이 세 가지 기능은 데이터베이스의 구조 설계, 데이터 처리, 보안 관리의 모든 단계를 포함한다. 

현재 MySQL, PostgreSQL, SQLite와 같은 많은 데이터베이스 시스템이 SQL을 지원하며, 효율적이고 안정적인 데이터 관리를 가능하게 한다.

#### 1.1 DDL (Data Definition Language)
DDL은 데이터베이스의 구조를 정의하고 관리하는 데 사용된다. 테이블, 인덱스, 뷰 등의 데이터베이스 객체를 생성, 수정, 삭제하는 명령어를 포함한다. 대표적인 명령어로는 `CREATE`, `ALTER`, `DROP` 등이 있다. 

예를 들어, `CREATE TABLE` 명령어를 사용해 새로운 테이블을 정의할 수 있다. 또한, `ALTER TABLE`을 사용해 기존 테이블에 새로운 열을 추가하거나 수정할 수 있다. 

DDL은 데이터베이스의 설계 초기 단계에서 중요하며, 데이터의 논리적 구조를 설정하는 역할을 한다. 데이터 타입과 제약 조건을 설정함으로써 데이터의 일관성과 무결성을 보장할 수 있다. 금융 데이터베이스에서는 주식 가격, 거래 기록, 경제 지표 등을 저장하는 테이블을 설계하는 데 자주 사용된다. 스키마 설계와 데이터 정의는 데이터베이스 성능과 데이터 분석의 정확성에 큰 영향을 미친다.

**Create Table**

가장 기초적인 작업은 테이블을 생성하는 것이다. 테이블은 하나의 엑셀 데이터를 생성하는 것과 같다고 보면 된다. 한 종목의 시계열 데이터는 대략 7개의 컬럼(날짜, 시가, 고가, 저가, 종가, 수정종가, 거래량)을 가진 table을 생성한다고 보면 편하다. DB데이터는 `sqlite3` 라이브러리의 `connect()` 함수를 통해 연결을 우선적으로 해 줘야 한다. `cursur()` method는 파이썬 내에서 SQL 명령문을 실행하도록 해 주는 역할을 한다.

In [19]:
DB_PATH = '../../data/price.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

In [20]:
type(conn) # Connection 

sqlite3.Connection

price.db에 `AAPL`이라는 table을 생성해 보자

In [21]:
cursor.execute(
    f"""
        CREATE table if not exists AAPL (
            date datetime,
            open numeric,
            high numeric,
            low numeric,
            close numeric,
            adj_close numeric,
            volume integer
        )
    """
)

DB에 접속하면 실제로 AAPL이라는 이름의 table이 생성되는 것을 확인할 수 있다.

**Modify Tables**

기존에 존재하는 테이블에 컬럼을 추가하여 저장하거나, 기존의 컬럼의 내용을 수정하고 싶을 수 있다. `ALTER` 명령문은 SQL에서 테이블 또는 데이터베이스 객체의 구조를 변경할 때 사용된다. 테이블에 새로운 열(Column)을 추가하거나 기존 열을 수정 또는 삭제하는 등의 작업을 수행할 수 있다. 또한, 제약 조건(Constraints) 추가, 데이터 타입 변경 등도 가능하다.

이미 생성한 테이블에 맨 끝쪽에 returns를 추가한다고 상상해 보자. 아래와 같이 SQL문을 작성할 수 있다.

In [22]:
cursor.execute(
    f"""
        ALTER TABLE AAPL 
        add returns real
    """
)

또는, 이미 추가된 컬럼을 삭제할 수도 있다.

In [25]:
cursor.execute(
    f"""
        ALTER TABLE AAPL 
        drop column returns;
    """
)

**Delete table**

table을 생성한 후 더이상 사용하지 않거나 쓸모 없다고 판단되는 경우 table 자체를 삭제할 수도 있다. `DROP` 명령문은 데이터베이스에서 특정 테이블을 완전히 삭제하는 데 사용된다. 이 명령어를 실행하면 테이블의 구조와 모든 데이터가 완전히 제거되며, 복구할 수 없다. 삭제된 테이블과 관련된 모든 인덱스, 제약 조건, 그리고 해당 테이블에 저장된 데이터도 함께 삭제된다.

In [26]:
cursor.execute(
    f"""
        DROP table AAPL
    """
)

이제 실제로 테이블을 여러 개 생성하고 적재 해 보자. 아래의 반복문을 통해 여러 테이블들을 쉽게 생성할 수 있다.

In [28]:
from tqdm import tqdm

TABLE_NAMES = {
    'SP500':'^SPX',
    'NASDAQ':'^IXIC',
    'RUSSELL':'^RUT',
    'DOWJONES':'^DJI',
    'AMEX':'^XAX'
}

for table_name in tqdm(list(TABLE_NAMES.keys())) :
    cursor.execute(
        f"""
            CREATE TABLE IF NOT EXISTS {table_name} (
                date datetime,
                open numeric,
                high numeric,
                low numeric,
                close numeric,
                adj_close numeric,
                volume integer
            )
        """
    )

100%|██████████| 5/5 [00:00<00:00, 1523.87it/s]


#### 1.2 DML (Data Manipulation Language)

테이블을 생성하였으니, 알맞은 데이터를 적재할 차례이다. DML은 데이터베이스에 저장된 데이터를 조회하거나 수정, 삽입, 삭제하는 작업을 수행한다. **가장 자주 사용되는 SQL 기능**으로, 데이터를 직접 다루기 때문에 **실시간 데이터 분석과 관리에서 중요**하다. 

대표적인 명령어로는 데이터를 삽입하는 `INSERT`, 데이터를 검색하는 `SELECT`, 데이터를 수정하는 `UPDATE`, 데이터를 삭제하는 `DELETE`가 있다. 예를 들어, `SELECT` 명령어를 사용해 특정 주식의 일별 가격을 조회할 수 있다. 혹은, 전체 가격 테이블에서 특정 종목의 가격 전체의 열을 불러올 수도 있다.

주로 데이터의 상태를 변경하거나 원하는 데이터를 추출하는 데 사용되며, 데이터베이스 사용자의 주요 인터페이스 역할을 한다. 금융 데이터에서는 특정 날짜의 주식 가격을 조회하거나, 거래 내역을 삽입하고 수정하는 데 활용된다. DML은 데이터를 효율적으로 다룰 수 있도록 설계되어 있어, 대규모 데이터베이스에서도 빠르고 정확하게 작동한다.

In [30]:
for table_name in tqdm(list(TABLE_NAMES.keys())) :
    data = yf.download(
        TABLE_NAMES[table_name],
        start = START_DATE,
        progress = False,
        interval = '1d'
    )
    data = data.reset_index()
    data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
    
    for _, row in data.iterrows():  # Use iterrows to iterate through each row
        cursor.execute( # SQL Query Statement
            f"""INSERT OR IGNORE INTO {table_name}(
                date, open, high, low, close, adj_close, volume
            )
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """, 
            (
                row['Date'].strftime('%Y-%m-%d'), 
                row['Open'], 
                row['High'], 
                row['Low'], 
                row['Close'], 
                row['Adj Close'], 
                row['Volume']
            )
        )
        conn.commit()

100%|██████████| 5/5 [00:16<00:00,  3.31s/it]


#### 1.3 DCL (Data Control Language)

DCL은 데이터베이스에 대한 접근 권한을 관리하고 보안을 유지하는 데 사용된다. 대표적인 명령어로는 권한을 부여하는 `GRANT`와 권한을 회수하는 `REVOKE`가 있다. 예를 들어, 특정 사용자에게 주식 데이터베이스의 읽기 권한만 부여하거나, 수정 권한을 제한할 수 있다. 

DCL은 데이터베이스의 무단 접근을 방지하고, 데이터의 기밀성을 보장하는 데 중요한 역할을 한다. 금융 데이터베이스에서는 민감한 데이터에 대한 접근을 통제해 데이터 유출이나 보안 사고를 예방할 수 있다. 또한, 데이터베이스 관리자는 사용자의 역할(Role)을 설정하고, 각 역할에 적합한 권한을 할당할 수 있다. 따라서 DCL은 데이터 무결성과 신뢰성을 유지하는 데 필수적이다. 데이터 보호가 중요한 환경에서는 DCL 명령어를 활용한 권한 관리를 정기적으로 점검해야 한다.

여기서는 데이터 적재 및 관리와 거리가 멀기 때문에 따로 다루지는 않도록 한다.